In [1]:
# Import the necessary functions
import logging
# Set the logging level to WARNING to suppress INFO messages
logging.basicConfig(level=logging.WARNING)
import transformers

#logging.set_verbosity_error()
# Disable specific warnings
transformers.logging.set_verbosity_error()
from inference_utils import load_model_and_tokenizer, generate_predictions, setup_device
import json #, jsonlines

from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer
import torch

from tqdm import tqdm
import re

/home/s220331/.conda/envs/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/s220331/GROK/Thesis/transformers/src/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/s220331/GROK/Thesis/transformers/src/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/s220331/GROK/Thesis/transformers/src/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytr

In [2]:
# Generate predictions
# Define the parameters for generation
max_length = 10  # Adjust the max_length as needed
num_return_sequences = 1  # Adjust the number of return sequence

#predictions = generate_predictions(model, tokenizer, input_texts, device, max_length=max_length, num_return_sequences=num_return_sequences)
def predict(model, tokenizer, input_texts, device, max_length=max_length, num_return_sequences=num_return_sequences):
    predictions = generate_predictions(model, tokenizer, input_texts, device, max_length=max_length, num_return_sequences=num_return_sequences)
    # Print the predictions
    for i, prediction in enumerate(predictions):
        print(f"Input: {input_texts[i]}")
        print(f"Prediction: {prediction}")

In [3]:
# To chose the "B" that compare 2 times both in ID and OOD**
def chose_B(range_):
    list_ID=[]
    list_OOD=[]
    list_inferred_ID=[]
    list_inferred_OOD=[]

    count=0
    for i in range(range_):
        target = f'<e_{i}></a>'
        t_print=f'<e_{i}>'
        filtered_texts = [entry['target_text'] for entry in d['id_atomic'] if entry['target_text'].endswith(target)]

        filtered_texts_OOD = [entry['target_text'] for entry in d['ood_atomic'] if entry['target_text'].endswith(target)]
        if count <=30:
            if len(filtered_texts) >= 2 and len(filtered_texts_OOD) >= 2:

                filtered_texts_2_hop = [entry['target_text'] for entry in d['id_atomic'] if entry['target_text'].startswith(t_print)]

                filtered_texts_OOD_2_hop = [entry['target_text'] for entry in d['ood_atomic'] if entry['target_text'].startswith(t_print)]

                # Extract the part before <e_57></a> from the first element
                #atomic_part_1_id = filtered_texts[0].split(target)[0]
                #print("atomic_id", atomic_part_1_id)
                #atomic_part_1_ood = str(filtered_texts_OOD[0].split(target)[0])

                #inferred_OOD = [entry['target_text'] for entry in d['test_inferred_ood']
                #                    if entry['type'] == 'test_inferred_ood' and entry['input_text'].startswith(atomic_part_1_ood)]

                #inferred_ID = [entry['target_text'] for entry in d['test_inferred_iid']
                #                    if entry['type'] == 'test_inferred_iid' and entry['input_text'].startswith(atomic_part_1_id)]

                if len(filtered_texts_2_hop) >= 1 and len(filtered_texts_OOD_2_hop) >= 1:
                    

                    list_ID.append(filtered_texts)
                    list_OOD.append(filtered_texts_OOD)
                    list_inferred_ID.append(filtered_texts_2_hop)
                    list_inferred_OOD.append(filtered_texts_OOD_2_hop,)

                    if count <=3:
                        print("-------Target :", t_print,  "     ----------------------------------------------------------\n")
                        print("ID",filtered_texts   ,"\n")
                        print("OOD",filtered_texts_OOD   ,"\n \n")

                        #print("Inferred _ ID", inferred_ID)
                        #print("Inferred _ OOD", inferred_OOD)
                        # for the second hop

                        count+=1
                        print("2- hop  ID", filtered_texts_2_hop)
                        print("2nd hop OOD", filtered_texts_OOD_2_hop, "\n \n \n")
                        
    return list_ID, list_OOD, list_inferred_ID, list_inferred_OOD


In [4]:
def return_rank(hd, word_embedding_, token, metric='dot', token_list=None):
    """
hd: Hidden states or output from a neural network.
word_embedding_: Embedding matrix for words. (matrix that convert words to their embedding representation)
token: The specific token (word) for which we want to find the rank.
metric: The similarity metric to use ('dot' for dot product, 'cos' for cosine similarity).
token_list: Optional list of tokens to consider for ranking."""

    if metric == 'dot':
        word_embedding = word_embedding_
    elif metric == 'cos':
        word_embedding = F.normalize(word_embedding_, p=2, dim=1)
    else:
        assert False
    #Compute the similarity scores (logits) between the hidden states (hd) and the word embeddings using matrix multiplication.
    logits_ = torch.matmul(hd, word_embedding.T)  # a vlaue higher if he similarity with the analyzed "word" is higehr

    rank = [] 
    for j in range(len(logits_)):
        log = logits_[j].cpu().numpy()
        if token_list is None:
            temp = [[i, log[i]] for i in range(len(log))]
        else:
            temp = [[i, log[i]] for i in token_list]
        temp.sort(key=lambda var: var[1], reverse=True)
        rank.append([var[0] for var in temp].index(token))
    return rank


In [38]:
import torch
import torch.nn.functional as F
from torch.nn import LayerNorm

def return_rank_norm_layer(hd, word_embedding_, token, metric='dot', token_list=None, layer_norm=None):
    """
    Returns the rank of a given token based on similarity between hidden states and word embeddings.

    Args:
        hd (torch.Tensor): Hidden states of shape (batch_size, hidden_dim).
        word_embedding_ (torch.Tensor): Word embedding matrix of shape (vocab_size, hidden_dim).
        token (int): Target token index whose rank is to be found.
        metric (str): Similarity metric ('dot' or 'cos'). Defaults to 'dot'.
        token_list (list, optional): List of token indices to consider. Defaults to None.
        layer_norm (LayerNorm, optional): LayerNorm instance for final layer normalization. 
                                          If None, no normalization is applied. Defaults to None.

    Returns:
        list: Ranks of the target token for each instance in the batch.
    """

    # Apply final layer normalization if provided
    if layer_norm is not None:
        hd = layer_norm(hd)

    # Normalize word embeddings if metric is cosine similarity
    if metric == 'cos':
        word_embedding = F.normalize(word_embedding_, p=2, dim=1)
    elif metric == 'dot':
        word_embedding = word_embedding_
    else:
        raise ValueError("Invalid metric. Use 'dot' or 'cos'.")

    # Compute logits by projecting hidden states onto the embedding matrix
    logits_ = torch.matmul(hd, word_embedding.T)

    # Calculate rank of the target token
    rank = []
    for log in logits_:
        log_np = log.cpu().numpy()
        candidates = [[i, log_np[i]] for i in (token_list if token_list is not None else range(len(log_np)))]
        candidates.sort(key=lambda var: var[1], reverse=True)
        rank.append([var[0] for var in candidates].index(token))

    return rank


In [ ]:
#dataset = "/home/s220331/GROK/Thesis/data/composition_SMALL.200.20.12.6"
dataset="/home/s220331/GROK/Thesis/data/composition.2000.200.12.6"
model_path = "/dtu-compute/s220331/composition/outputs_BIG_new/checkpoint-1500000/"
#"/dtu-compute/s220331/composition/outputs_BIG_new/checkpoint-1500000/" 

target_layer = 8


# model set up
#device = torch.device('cuda:5')
device = setup_device()
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
word_embedding = model.lm_head.weight.data
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id

In [5]:
#test case to remove
dataset = "/home/s220331/GROK/Thesis/data/composition_SMALL.200.20.12.6"
#dataset="/home/s220331/GROK/Thesis/data/composition.2000.200.12.6"  #"/dtu-compute/s220331/composition/outputs_BIG_new/checkpoint-1500000/"
model_path = "/scratch/davide/model_paper/outputs_SMALL/checkpoint-1500000/" # model normal on small dataset wuth chosen checkpoints!
target_layer = 8


# model set up
#device = torch.device('cuda:5')
device = setup_device()
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
word_embedding = model.lm_head.weight.data
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id

In [6]:


# set data

all_atomic = set()     # (h,r,t)
atomic_dict = dict()   # (h,r) -> t
with open(dataset+"/train.json") as f:  # from the correct data or data_MIO !!!
    train_items = json.load(f)
for item in tqdm(train_items):
    temp = item['target_text'].strip("><").split("><")
    if len(temp) != 4:
        continue
    h,r,t = temp[:3]
    atomic_dict[(h,r)] = t
    all_atomic.add((h,r,t))

id_atomic = set()
for item in tqdm(train_items):
    temp = item['target_text'].strip("><").split("><")
    if len(temp) == 4:
        continue
    h, r1, r2, t = temp[:4]
    b = atomic_dict[(h, r1)]
    assert atomic_dict[(b, r2)] == t
    id_atomic.add((h,r1,b))
    id_atomic.add((b,r2,t))

ood_atomic = all_atomic - id_atomic
print("# id_atomic, # ood_atomic:", len(id_atomic), len(ood_atomic))

# smart way to save all the train
with open(dataset+"/test.json") as f:
    pred_data = json.load(f)
d = dict()
for item in pred_data:
    t = item['type']
    if t not in d:
        d[t] = []
    d[t].append(item)

100%|██████████| 51880/51880 [00:00<00:00, 627269.72it/s]

# id_atomic, # ood_atomic: 3800 200


**To chose the "B" that compare 2 times both in ID and OOD**

In [7]:
a= chose_B(199)
#a[1][0]



-------Target : <e_11>      ----------------------------------------------------------

ID ['<e_150><r_11><e_11></a>', '<e_140><r_19><e_11></a>', '<e_128><r_3><e_11></a>'] 

OOD ['<e_147><r_4><e_11></a>', '<e_9><r_16><e_11></a>'] 
 

2- hop  ID ['<e_11><r_11><e_88></a>']
2nd hop OOD ['<e_11><r_1><e_141></a>'] 
 
 

-------Target : <e_12>      ----------------------------------------------------------

ID ['<e_38><r_7><e_12></a>', '<e_195><r_5><e_12></a>', '<e_122><r_8><e_12></a>'] 

OOD ['<e_136><r_2><e_12></a>', '<e_1><r_16><e_12></a>'] 
 

2- hop  ID ['<e_12><r_4><e_21></a>']
2nd hop OOD ['<e_12><r_2><e_5></a>'] 
 
 

-------Target : <e_47>      ----------------------------------------------------------

ID ['<e_117><r_4><e_47></a>', '<e_32><r_8><e_47></a>', '<e_130><r_3><e_47></a>', '<e_79><r_3><e_47></a>'] 

OOD ['<e_105><r_9><e_47></a>', '<e_6><r_1><e_47></a>'] 
 

2- hop  ID ['<e_47><r_15><e_180></a>']
2nd hop OOD ['<e_47><r_7><e_13></a>'] 
 
 

-------Target : <e_114>      -----

In [39]:
query_change_OOD ="<e_147><r_4><r_1>" #"<e_181><r_6><r_5>"# "<e_147><r_4><r_1>"# "<e_14><r_12><r_13>"       # 2 hop ['<e_11><r_1><e_141></a>'] 

def run_query(query_change_OOD, ood=False):
    # inser here the 2-hop query (like ood) from whihc extract the first hop-- "b" activations
    # usato "<e_147><r_4><r_1>"
    h_1_ood,n_r_1_ood,r_1_ood = query_change_OOD.strip("><").split("><") #"e_140","r_19","r_11"             #query.strip("><").split("><")
    b_1_ood = atomic_dict[(h_1_ood ,n_r_1_ood)]
    t_1_ood = atomic_dict[(b_1_ood, r_1_ood)]
    list_b_t=[]

    print("b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa ",b_1_ood)
    print("tail:",t_1_ood)


    # Tokenizing the Query:
    decoder_temp_1_OOD = tokenizer([query_change_OOD], return_tensors="pt", padding=True)
    decoder_input_ids_1_OOD, decoder_attention_mask_1_OOD = decoder_temp_1_OOD["input_ids"], decoder_temp_1_OOD["attention_mask"]
    decoder_input_ids_1_OOD, decoder_attention_mask_1_OOD = decoder_input_ids_1_OOD.to(device), decoder_attention_mask_1_OOD.to(device)


    with torch.no_grad():
        # here the same model but with different input !! 
        outputs_1_OOD = model(
            input_ids=decoder_input_ids_1_OOD,
            attention_mask=decoder_attention_mask_1_OOD,
            output_hidden_states=True
        )
    # hidden state of the 1_id (which will be inserted in the normal run)
    all_hidden_states_1_OOD = outputs_1_OOD['hidden_states']

    #################### just to quick check everything is right

    res_dict_b_1_OOD = dict()
    target_layer_b = 4
    target_layer_t = 8
    # ("<"+ b +">")  is just for reconstructing ("<b>") from "b"
    rank_before_1_OOD = return_rank_norm_layer(all_hidden_states_1_OOD[target_layer_b][0, :, :], word_embedding, tokenizer("<"+ b_1_ood +">")['input_ids'][0])[-2]
    res_dict_b_1_OOD['rank_before, layer 5 b'] = rank_before_1_OOD

    # laste layer search for t
    rank_before_1_OOD_t = return_rank_norm_layer(all_hidden_states_1_OOD[target_layer_t][0, :, :], word_embedding, tokenizer("<"+ t_1_ood +">")['input_ids'][0])[-1]
    res_dict_b_1_OOD['rank_before, layer 8 search t'] = rank_before_1_OOD_t
    list_b_t.append((rank_before_1_OOD, rank_before_1_OOD_t))
    if ood:
        print("OOD")
        print(" B 1_OOD",res_dict_b_1_OOD)
        print("as expected in the case of OOD the B is corrctly found at layer 5 but not T at layer 8")
    else:
        print("ID")
        print(" B 1_OOD",res_dict_b_1_OOD)
        print("as expected in the case of ID the B is corrctly found at layer 5 and T at layer 8")

    return all_hidden_states_1_OOD , b_1_ood ,t_1_ood ,list_b_t



In [40]:
res_dict_OOD_intervention= dict()
def replace_hidden_state(hidden_to_insert,b, all_hidden_states_to_change,t):
    list_intervention_results = []
    target_layer_intervention = 4
    target_layer_final = 8
    # juest rename for more standardized structure
    all_hidden_states_ctft_1 = hidden_to_insert

    # we are analysing  with replacing only specific hidden state once a time
    for layer_to_intervene in range(target_layer_intervention , target_layer_intervention +1):
        print(layer_to_intervene)
        hidden_states_1 = all_hidden_states_to_change[layer_to_intervene].clone()    # cloniamo gli hidden del normal run
        hidden_states_ctft_1 = all_hidden_states_ctft_1[layer_to_intervene]  # copiamo quelli del run che inseriremo
    #r_2  r
        #r_2_ood = all_hidden_states_1_OOD_test[layer_to_intervene]

        # intervene
        hidden_states_1[0, 0, :] = hidden_states_ctft_1[0, 0, :]   #insereiamo gli hidden state che vogliamo
        hidden_states_1[0, 1, :] = hidden_states_ctft_1[0, 1, :]
        #hidden_states_1[0, 2, :] = hidden_states_ctft_1[0, 2, :]  # prova ad inserire r_2 di ood
        #hidden_states_1[0, 2, :]= r_11_id
        #hidden_states_1[0, 2, :]= r_2_ood[0, 2, :] # this is a r(second hop) coming from another ood (with not even a;r_1 the same. This prove that r_2 does not 
        # store calculation in the first 5 layers according to paper results.)
        

        rank_middle_1 = return_rank_norm_layer(hidden_states_1[0, :, :], word_embedding, tokenizer("<"+b+">")['input_ids'][0])[-2]
        res_dict_OOD_intervention["insertion at layer"+str(layer_to_intervene), "B"] = rank_middle_1
        #                
        with torch.no_grad():
            for i in range(layer_to_intervene, target_layer_final):
                f_layer_1 = model.transformer.h[i]  # current layer
                print("layer aggiornato:",i)
                # attention mechanism 
                residual_1 = hidden_states_1       #store the hidden state in residual
                # Apply layer normalization to hidden_states.
                hidden_states_1 = f_layer_1.ln_1(hidden_states_1)
                attn_output_1 = f_layer_1.attn(hidden_states_1)[0] 
                hidden_states_1 = attn_output_1 + residual_1
                # mlp  = Feed-Forward Network:
                residual_1 = hidden_states_1
                hidden_states_1 = f_layer_1.ln_2(hidden_states_1)
                feed_forward_hidden_states_1 = f_layer_1.mlp.c_proj(f_layer_1.mlp.act(f_layer_1.mlp.c_fc(hidden_states_1)))
                hidden_states_1 = residual_1 + feed_forward_hidden_states_1
            # final ln
            hidden_states_1 = model.transformer.ln_f(hidden_states_1)
        # print("--------")
        rank_after_1 = return_rank(hidden_states_1[0, :, :], word_embedding, tokenizer("<"+t+">")['input_ids'][0])[-1]
        res_dict_OOD_intervention["Out-tput layer"+str(target_layer_final), "t"] = rank_after_1
        list_intervention_results.append(rank_middle_1)
        list_intervention_results.append(rank_after_1)
        print("intervention results",res_dict_OOD_intervention)
    print( list_intervention_results)
    return list_intervention_results


## **Intervention with OOD insertion**

What: Insert in the normal run (ID run) the 5 layer hydden layers coming from a OOD run representing the same Bridge entity and r_2 relation.
It has be seen that the model still predict B (as expected as also the OOD predict the B) but also the t. In other words, does not seem that the model store and use information about how he gets to the Bridge. Given one Bridge he is able to secondo hop if the second hop is in ID.

In [41]:
atomic_to_chose = chose_B(199)

-------Target : <e_11>      ----------------------------------------------------------

ID ['<e_150><r_11><e_11></a>', '<e_140><r_19><e_11></a>', '<e_128><r_3><e_11></a>'] 

OOD ['<e_147><r_4><e_11></a>', '<e_9><r_16><e_11></a>'] 
 

2- hop  ID ['<e_11><r_11><e_88></a>']
2nd hop OOD ['<e_11><r_1><e_141></a>'] 
 
 

-------Target : <e_12>      ----------------------------------------------------------

ID ['<e_38><r_7><e_12></a>', '<e_195><r_5><e_12></a>', '<e_122><r_8><e_12></a>'] 

OOD ['<e_136><r_2><e_12></a>', '<e_1><r_16><e_12></a>'] 
 

2- hop  ID ['<e_12><r_4><e_21></a>']
2nd hop OOD ['<e_12><r_2><e_5></a>'] 
 
 

-------Target : <e_47>      ----------------------------------------------------------

ID ['<e_117><r_4><e_47></a>', '<e_32><r_8><e_47></a>', '<e_130><r_3><e_47></a>', '<e_79><r_3><e_47></a>'] 

OOD ['<e_105><r_9><e_47></a>', '<e_6><r_1><e_47></a>'] 
 

2- hop  ID ['<e_47><r_15><e_180></a>']
2nd hop OOD ['<e_47><r_7><e_13></a>'] 
 
 

-------Target : <e_114>      -----

In [42]:


# take 1 from 1_id and 1 from inferred to construct the query
ID_ID_list = []
OOD_OOD_list = []
OOD_intervention_in_ID= []
for i in range(0,7):
    _1_id = atomic_to_chose[0][i]  # list with all the
    _1_ood = atomic_to_chose[1][i]
    _1_id_inferred = atomic_to_chose[2][i]
    _1_ood_inferred = atomic_to_chose[3][i]

    text_1 = _1_id[1]
    match = re.match(r'^(<e_\d+><r_\d+>)', text_1)
    if match:
        result_1_id = match.group(1)
    text_2= _1_id_inferred[0]
    match = re.match(r'^<e_\d+>(<r_\d+>)', text_2)
    if match:
        result2_id = match.group(1)

    text_3 = _1_ood[0]
    match = re.match(r'^(<e_\d+><r_\d+>)', text_3)
    if match:
        result_1_ood = match.group(1)
    text_4= _1_ood_inferred[0]
    match = re.match(r'^<e_\d+>(<r_\d+>)', text_4)
    if match:
        result2_ood = match.group(1)

    id_id = result_1_id + result2_id
    ood_ood = result_1_ood + result2_ood
    print("ID_ID---------------------------------------------------------------------------", id_id)   
    print("OOD_OOD------------------------------------------------------------------------------------", ood_ood)
    normal_run_ID_ID =run_query(id_id, ood=False)
    B_OOD_insert_activations =run_query(ood_ood, ood=True) 
    print("-------------------------------------------------------------------------------------------------------------------b",B_OOD_insert_activations[1])
    
    intervention=  replace_hidden_state(B_OOD_insert_activations[0],B_OOD_insert_activations[1],normal_run_ID_ID[0], normal_run_ID_ID[2])  # hidden to insert, b, (all hidden states) normal run, t
    
    ID_ID_list.append(normal_run_ID_ID[3])
    OOD_OOD_list.append(B_OOD_insert_activations[3])
    OOD_intervention_in_ID.append(intervention)  



ID_ID--------------------------------------------------------------------------- <e_140><r_19><r_11>
OOD_OOD------------------------------------------------------------------------------------ <e_147><r_4><r_1>
b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa  e_11
tail: e_88
ID
 B 1_OOD {'rank_before, layer 5 b': 0, 'rank_before, layer 8 search t': 0}
as expected in the case of ID the B is corrctly found at layer 5 and T at layer 8
b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa  e_11
tail: e_141
OOD
 B 1_OOD {'rank_before, layer 5 b': 0, 'rank_before, layer 8 search t': 156}
as expected in the case of OOD the B is corrctly found at layer 5 but not T at layer 8
-------------------------------------------------------------------------------------------------------------------b e_11
4
layer aggiornato: 4
layer aggiornato: 5
layer aggiornato: 6
layer aggiornato: 7
intervention results {('insertion at layer4', 'B'): 0, ('Out-tput layer8', 't'): 0}
[0, 0]

In [43]:
len(atomic_to_chose[0])

9

In [44]:
OOD_OOD_list

[[(0, 156)],
 [(0, 83)],
 [(0, 118)],
 [(0, 43)],
 [(0, 114)],
 [(0, 64)],
 [(0, 79)]]

In [45]:
OOD_intervention_in_ID

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

In [46]:
ID_ID_list

[[(0, 0)], [(0, 0)], [(0, 0)], [(0, 0)], [(0, 0)], [(0, 0)], [(0, 0)]]

# PURE B fro layer 0 to lyer 4

In [89]:
res_dict_OOD_intervention_PURE__B= dict()
def replace_hidden_state_B(hidden_to_insert,b, all_hidden_states_to_change,t):
    list_intervention_results = []
    target_layer_intervention = 4
    target_layer_final = 8
    target_B=0
    # juest rename for more standardized structure
    all_hidden_states_ctft_1 = hidden_to_insert

    # we are analysing  with replacing only specific hidden state once a time
    for layer_to_intervene in range(target_layer_intervention , target_layer_intervention +1):
        print(layer_to_intervene)
        hidden_states_1 = all_hidden_states_to_change[layer_to_intervene].clone()    # cloniamo gli hidden del normal run
        hidden_states_ctft_1 = all_hidden_states_ctft_1[target_B]  # copiamo quelli del run che inseriremo
    #r_2  r
        #r_2_ood = all_hidden_states_1_OOD_test[layer_to_intervene]

        # intervene
        #hidden_states_1[0, 0, :] = hidden_states_ctft_1[0, 0, :]   #insereiamo gli hidden state che vogliamo
        hidden_states_1[0, 1, :] = hidden_states_ctft_1[0, 2, :]
        hidden_states_1[0, 2, :] = hidden_states_ctft_1[0, 1, :]  # prova ad inserire r_2 di ood
        #hidden_states_1[0, 2, :]= r_11_id
        #hidden_states_1[0, 2, :]= r_2_ood[0, 2, :] # this is a r(second hop) coming from another ood (with not even a;r_1 the same. This prove that r_2 does not 
        # store calculation in the first 5 layers according to paper results.)
        

        rank_middle_1 = return_rank_norm_layer(hidden_states_1[0, :, :], word_embedding, tokenizer("<"+b+">")['input_ids'][0])[-2]
        res_dict_OOD_intervention["insertion at layer"+str(layer_to_intervene), "B"] = rank_middle_1
        #                
        with torch.no_grad():
            for i in range(layer_to_intervene, target_layer_final):
                f_layer_1 = model.transformer.h[i]  # current layer
                print("layer aggiornato:",i)
                # attention mechanism 
                residual_1 = hidden_states_1       #store the hidden state in residual
                # Apply layer normalization to hidden_states.
                hidden_states_1 = f_layer_1.ln_1(hidden_states_1)
                attn_output_1 = f_layer_1.attn(hidden_states_1)[0] 
                hidden_states_1 = attn_output_1 + residual_1
                # mlp  = Feed-Forward Network:
                residual_1 = hidden_states_1
                hidden_states_1 = f_layer_1.ln_2(hidden_states_1)
                feed_forward_hidden_states_1 = f_layer_1.mlp.c_proj(f_layer_1.mlp.act(f_layer_1.mlp.c_fc(hidden_states_1)))
                hidden_states_1 = residual_1 + feed_forward_hidden_states_1
            # final ln
            hidden_states_1 = model.transformer.ln_f(hidden_states_1)
        # print("--------")
        rank_after_1 = return_rank_norm_layer(hidden_states_1[0, :, :], word_embedding, tokenizer("<"+t+">")['input_ids'][0])[-1]
        res_dict_OOD_intervention["Out-tput layer"+str(target_layer_final), "t"] = rank_after_1
        list_intervention_results.append(rank_middle_1)
        list_intervention_results.append(rank_after_1)
        print("intervention results",res_dict_OOD_intervention)
    print( list_intervention_results)
    return list_intervention_results


In [90]:
query_change_OOD ="<e_147><r_4><r_1>" #"<e_181><r_6><r_5>"# "<e_147><r_4><r_1>"# "<e_14><r_12><r_13>"       # 2 hop ['<e_11><r_1><e_141></a>'] 

def run_query_B(query_change_OOD, ood=False):
    # inser here the 2-hop query (like ood) from whihc extract the first hop-- "b" activations
    # usato "<e_147><r_4><r_1>"
    h_1_ood,n_r_1_ood,r_1_ood = query_change_OOD.strip("><").split("><") #"e_140","r_19","r_11"             #query.strip("><").split("><")
    b_1_ood = atomic_dict[(h_1_ood ,n_r_1_ood)]
    t_1_ood = atomic_dict[(b_1_ood, r_1_ood)]
    list_b_t=[]

    print("b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa ",b_1_ood)
    print("tail:",t_1_ood)
    
    query_B_layer_for_layer_0=  "<"+ h_1_ood +">" +"<"+ n_r_1_ood +">"+ "<"+ b_1_ood +">"

    # Tokenizing the Query:
    decoder_temp_1_OOD = tokenizer([query_B_layer_for_layer_0], return_tensors="pt", padding=True)
    decoder_input_ids_1_OOD, decoder_attention_mask_1_OOD = decoder_temp_1_OOD["input_ids"], decoder_temp_1_OOD["attention_mask"]
    decoder_input_ids_1_OOD, decoder_attention_mask_1_OOD = decoder_input_ids_1_OOD.to(device), decoder_attention_mask_1_OOD.to(device)


    with torch.no_grad():
        # here the same model but with different input !! 
        outputs_1_OOD = model(
            input_ids=decoder_input_ids_1_OOD,
            attention_mask=decoder_attention_mask_1_OOD,
            output_hidden_states=True
        )
    # hidden state of the 1_id (which will be inserted in the normal run)
    all_hidden_states_1_OOD = outputs_1_OOD['hidden_states']

    #################### just to quick check everything is right

    res_dict_b_1_OOD = dict()
    target_layer_b = 4
    target_layer_t = 8
    # ("<"+ b +">")  is just for reconstructing ("<b>") from "b"
    rank_before_1_OOD = return_rank_norm_layer(all_hidden_states_1_OOD[target_layer_b][0, :, :], word_embedding, tokenizer("<"+ b_1_ood +">")['input_ids'][0])[-2]
    res_dict_b_1_OOD['rank_before, layer 5 b'] = rank_before_1_OOD

    # laste layer search for t
    rank_before_1_OOD_t = return_rank_norm_layer(all_hidden_states_1_OOD[target_layer_t][0, :, :], word_embedding, tokenizer("<"+ t_1_ood +">")['input_ids'][0])[-1]
    res_dict_b_1_OOD['rank_before, layer 8 search t'] = rank_before_1_OOD_t
    list_b_t.append((rank_before_1_OOD, rank_before_1_OOD_t))
    if ood:
        print("OOD")
        print(" B 1_OOD",res_dict_b_1_OOD)
        print("as expected in the case of OOD the B is corrctly found at layer 5 but not T at layer 8")
    else:
        print("ID")
        print(" B 1_OOD",res_dict_b_1_OOD)
        print("as expected in the case of ID the B is corrctly found at layer 5 and T at layer 8")

    return all_hidden_states_1_OOD , b_1_ood ,t_1_ood ,list_b_t



In [91]:


# take 1 from 1_id and 1 from inferred to construct the query
ID_ID_list_P = []
OOD_OOD_list_P = []
OOD_intervention_in_ID_P= []
for i in range(0,7):
    _1_id = atomic_to_chose[0][i]  # list with all the
    _1_ood = atomic_to_chose[1][i]
    _1_id_inferred = atomic_to_chose[2][i]
    _1_ood_inferred = atomic_to_chose[3][i]

    text_1 = _1_id[0]
    match = re.match(r'^(<e_\d+><r_\d+>)', text_1)
    if match:
        result_1_id = match.group(1)

    text_1_1 = _1_id[1]
    match_1 = re.match(r'^(<e_\d+><r_\d+>)', text_1_1)
    if match_1:
        result_1_id_1 = match_1.group(1)

    
    text_2= _1_id_inferred[0]
    match = re.match(r'^<e_\d+>(<r_\d+>)', text_2)
    if match:
        result2_id = match.group(1)

    text_3 = _1_ood[0]
    match = re.match(r'^(<e_\d+><r_\d+>)', text_3)
    if match:
        result_1_ood = match.group(1)
    text_4= _1_ood_inferred[0]
    match = re.match(r'^<e_\d+>(<r_\d+>)', text_4)
    if match:
        result2_ood = match.group(1)

    id_id = result_1_id + result2_id

    id_id_1 = result_1_id_1 + result2_id
    ood_ood = result_1_ood + result2_ood
    print("ID_ID---------------------------------------------------------------------------", id_id)   
    print("OOD_OOD------------------------------------------------------------------------------------", ood_ood)
    normal_run_ID_ID =run_query(id_id, ood=False)
    #B_OOD_insert_activations =run_query(ood_ood, ood=True) 
    normal_2 =run_query_B(id_id_1 , ood=True)
    print("-------------------------------------------------------------------------------------------------------------------b",B_OOD_insert_activations[1])
    
    intervention=  replace_hidden_state_B(normal_2[0],normal_2[1],normal_run_ID_ID[0], normal_run_ID_ID[2])  # hidden to insert, b, (all hidden states) normal run, t
    #intervention=  replace_hidden_state_B(normal_2[0],normal_2[1],normal_run_ID_ID[0], normal_run_ID_ID[2])  # hidden to insert, b, (all hidden states) normal run, t
    
    ID_ID_list_P.append(normal_run_ID_ID[3])
    OOD_OOD_list_P.append(normal_2[3])
    OOD_intervention_in_ID_P.append(intervention)  



ID_ID--------------------------------------------------------------------------- <e_150><r_11><r_11>
OOD_OOD------------------------------------------------------------------------------------ <e_147><r_4><r_1>
b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa  e_11
tail: e_88
ID
 B 1_OOD {'rank_before, layer 5 b': 0, 'rank_before, layer 8 search t': 0}
as expected in the case of ID the B is corrctly found at layer 5 and T at layer 8
b: quaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa  e_11
tail: e_88
OOD
 B 1_OOD {'rank_before, layer 5 b': 0, 'rank_before, layer 8 search t': 107}
as expected in the case of OOD the B is corrctly found at layer 5 but not T at layer 8
-------------------------------------------------------------------------------------------------------------------b e_151
4
layer aggiornato: 4
layer aggiornato: 5
layer aggiornato: 6
layer aggiornato: 7
intervention results {('insertion at layer4', 'B'): 0, ('Out-tput layer8', 't'): 127}
[0, 

In [92]:
ID_ID_list_P

[[(0, 0)], [(0, 0)], [(0, 0)], [(0, 0)], [(0, 0)], [(0, 0)], [(0, 0)]]

In [93]:
OOD_OOD_list_P

[[(0, 107)], [(0, 10)], [(0, 1)], [(0, 26)], [(0, 90)], [(0, 5)], [(0, 2)]]

In [94]:
OOD_intervention_in_ID_P

[[0, 127], [0, 119], [0, 17], [0, 77], [0, 117], [0, 13], [0, 2]]